In [1]:
import os
import bs4
from bs4 import BeautifulSoup as soup
import pandas as pd
import spacy
import re
import en_core_web_sm
import keras
import tensorflow
from pathlib import Path
from pathlib import PureWindowsPath
import scrapy
from dateparser.search import search_dates
import arrow
import datetime

Using TensorFlow backend.


In [2]:
# nlp = spacy.load('en')

In [3]:
def extract_html(input_html):
    with open(input_html) as inf:
        txt = inf.read()
    soup_out = soup(txt, "html.parser")
    print(input_html)
    return soup_out

In [4]:
def extract_firm(input_soup):
    temp = input_soup.find(id='article_participants')
    if temp.findAll('a')==[]:
        temp = input_soup.find(id='article_content')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.get('title')))   
        
    return None


In [5]:
def extract_ticker(input_soup):
    temp = input_soup.find(id='article_participants')
    if temp.findAll('a')==[]:
        temp = input_soup.find(id='article_content')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.contents[0].strip()))   
    return None

In [6]:
import dateparser
def extract_time(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.find_all('p'):
        temp_string=element.text.strip()
        try:
#             December 2, 2014 11:15 AM ET
            temp_string = re.match(string=temp_string,pattern='([a-zA-Z]+\s+[0-9]+,\s+[0-9]{4})[^ 0-9]?(\s+[0-9]{1,2}:[0-9]{2}\s+[APap]\.?[mM]\.?)')
            string1 = temp_string.group(1)
            string2 = temp_string.group(2)
            #         name_search.group(1).strip()
            string3 = string1 + string2
            string3 = re.sub(string=string3,pattern='\.',repl='')
            string3 = re.sub(string=string3,pattern='\s+',repl=' ')
#             print(string1,'##',string2,'##',string3)
            date_out = arrow.get(string3,'MMMM D, YYYY H:mm A')
        except:
            date_out=None        
            
        if date_out is not None:
            return date_out.isoformat()
    temp = input_soup.find(id='article_content')
    
    for element in temp.find_all('a'):
        temp_string=element.text.strip()
        temp_string = re.sub(string=temp_string,pattern='\s+',repl=' ')
        try:
            while  (element):
                try:
                    date_out = arrow.get(temp_string,'MMMM D, YYYY H:mm A')
                except:
                    date_out=None       
                if date_out is not None:
    #                 return temp_string
                    return date_out.isoformat()


                element = element.next_sibling
                temp_string = element
                temp_string = re.sub(string=temp_string,pattern='\s+',repl=' ')  
    #             print(temp_string.find('div'))
    #             print(temp_string.find('strong'))
        except:
            pass
    return None



In [7]:
def extract_quarter(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[0])
        return None
    except:
        temp = input_soup.find(id='article_participants')

        return None
    
def extract_year(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[1])
        return None
    except:
        return None

In [8]:
def test_analyst(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('strong'):
        if(re.search('analyst',element.contents[0],flags=re.IGNORECASE)):
            return(True)
    return(None)


In [9]:
def extract_analysts(input_soup):
    temp = input_soup.find(id='article_participants')
    test_string = []
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('analyst',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
            #print(element.contents[0].strip())
            #print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):
            #print('FOUND')
                    break
                if (pd.notnull(element.text) and element.text!=''):
                    test_string.append(element.contents[0].strip())    
            break
    return test_string

In [10]:
def extract_executives(input_soup):
    test_string = []
    temp = input_soup.find(id='article_participants')
    # print(soup_out)
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('executive',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
                # print(element.contents[0])
                # print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):            
                #print('FOUND')
                    break
                test_string.append(element.text.strip())
        
            break
    return test_string

In [11]:
def is_analyst_present(analyst_list):
    if analyst_list: 
        return True
    else: 
        return False

In [12]:
def extract_name (input_string,delim='-'):
    if not pd.isnull(input_string):
        name_search = re.search('(^[^-]*)-', input_string, re.IGNORECASE)
        try:
            return name_search.group(1).strip()
        except:
            return None        
    else:
        return None
def extract_title (input_string,delim='-'):
    if not pd.isnull(input_string):
        title_search = re.search('^[^-]*-(.*)', input_string, re.IGNORECASE)
        try:
            return title_search.group(1).strip()
        except:
            return None           
        else:
            return None
    else:
        return None

In [13]:
def extract_content(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_content')
        test_string = []
        # print(temp)
        for element in temp.find_all('strong'):
            if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                while  (1):
                    element = element.find_next()
        #           print(element.text.strip())
                    if element.find('strong') or \
                    element.find('div'):
                        break
                    test_string.append(element.text.strip())    
        return test_string
    else:
        return None

In [14]:
def extract_qanda(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_qanda')
        test_string = []
        # print(temp)
        if temp.find_all('span'):
            for element in temp.find_all('span'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())    
        else:                
            for element in temp.find_all('strong'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())      
            
        return test_string
    else:
        return None

In [15]:
nlp = en_core_web_sm.load()

def basic_wc(input_text):
    if not (input_text==[]):
#         decoded_txt=input_text.decode('utf-8')
        sent =  " ".join(str(x) for x in input_text)
        sent = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=sent)            
        doc = nlp(sent)
        return doc.__len__()
    else:
        return 0


In [16]:
def extract_chunks(input_text):
    sent =  " ".join(str(x) for x in input_text)
    #     doc = nlp(input_text)
    doc = nlp(sent)
    spans = list(doc.ents)  #+ list(doc.noun_chunks)
    out_list = []
    for span in spans:
        span.merge()
        out_list.append(span)
    return out_list

In [17]:
def extract_name_pair(name, input_text):
    out_list = []
    if not (input_text==[]):
#         s_out = extract_chunks(input_text)  
        sent =  " ".join(str(x) for x in input_text)
        #     doc = nlp(input_text)
        doc = nlp(sent)        
#         doc = nlp(input_text)
        spans = list(doc.ents)  #+ list(doc.noun_chunks)

        for item in spans:
            try:
                item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
                if (re.search(item,name,flags=re.IGNORECASE)):
                    out_list.append(item)

            except:
                pass
            
    return out_list
#     else:
#         return None

In [18]:
collection = "html"

# n_test=50
file_list = os.listdir(collection)
# for i in range(len(file_list)):
# for i in range(n_test):
#     print(file_list[i])
print(len(file_list))

38490


In [19]:
import csv
fieldnames = ['index','fname', 'status']
# output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/dates')
input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')
file_list = os.listdir(input_folder)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
print(len(file_list))
for i in range(len(file_list)):
    with open(input_folder / file_list[i]) as inf:
        if(re.search('html',file_list[i],flags=re.IGNORECASE)):
            out_list.append(file_list[i])
#             print(file_list[i])

FILE	ANALYST_Flag	FIRM	Quarter	Year
91979


In [ ]:
# out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
out_executives.drop(['text','executive','execs'],axis=1)\
.to_csv('analyst_out_foringestion.csv',sep=',', encoding = 'utf-8')
# out_executives.select({[}'file','quarter_yer'])

In [20]:
ls

 Volume in drive Z is Data
 Volume Serial Number is 647E-DA71

 Directory of Z:\cse315\textparsing

10/03/2018  03:06 PM    <DIR>          .
10/03/2018  03:06 PM    <DIR>          ..
10/03/2018  02:58 PM    <DIR>          .git
09/21/2018  06:40 PM    <DIR>          .ipynb_checkpoints
09/19/2018  01:35 PM    <DIR>          html
09/19/2018  01:35 PM                 0 log_extract.csv
09/19/2018  01:02 PM                 0 log_ner.csv
09/22/2018  05:02 AM            10,502 Named Entity Recognition Pipeline.ipynb
09/22/2018  01:12 AM            10,493 Named Entity Recognition Pipeline-Copy1.ipynb
09/22/2018  12:42 AM            10,502 Named Entity Recognition Pipeline-Copy2.ipynb
09/22/2018  01:28 AM            10,504 Named Entity Recognition Pipeline-Copy3.ipynb
09/19/2018  01:02 PM             9,337 output_0912.xlsx
09/21/2018  01:22 PM           267,033 Parse Conversation HTML.ipynb
09/19/2018  01:02 PM                58 README.md
09/19/2018  01:03 PM    <DIR>          sample_out
09/19/2

In [20]:
import csv
fieldnames = ['index','fname', 'status']
# output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/dates')
input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')

with open(output_folder / 'log_extract.csv', 'w') as csvfile:

    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    for ii in range(1,20000):
        try:
            total_rows = len(file_list)
            row_start = ii*1
            row_end = (ii*1)+1
            filename = 'exec_out_foringestion_'+str(row_start)+'.csv'

            out_list =[]

            for i in range(row_start,row_end):
                with open(input_folder / file_list[i]) as inf:
#                 with open(collection + '/' + file_list[i]) as inf:
                    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
                        out_list.append(file_list[i])

            out_pd = pd.DataFrame(out_list)
            out_pd.columns=['file']
            out_pd['text'] = out_pd.apply(lambda x: extract_html(input_folder / x['file']), axis=1)
            out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
            out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
            out_pd['datestamp'] = out_pd.apply(lambda x: extract_time(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)
            
            out_executives = out_pd                      
            out_executives.drop(['text'],axis=1).to_csv(output_folder / filename,sep=',', encoding = 'utf-8',header=False)
            logwriter.writerow([ii,out_executives.file.head(1).item(),'success'])
        except:
            print('exception: ', ii, ' ###')
            logwriter.writerow([ii,out_executives.file.head(1).item(),'fail'])
            
            pass

C:\Users\T.Santos18\Documents\workspace\textparsing\html\1.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\100.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\1000.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10000.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10001.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10002.html
exception:  7  ###
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10003.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10004.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10005.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10006.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10007.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10008.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10009.html
C:\Users\T.Santos18\Documents\workspace

C:\Users\T.Santos18\Documents\workspace\textparsing\html\10110.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10111.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10112.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10113.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10114.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10115.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10116.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10117.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10118.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10119.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\1012.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10121.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10122.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10123.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\10223.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10224.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10225.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10226.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10227.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10228.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10229.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\1023.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10230.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10231.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10232.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10233.html
exception:  261  ###
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10235.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10236.html
C:\Users\T.Santos18\Document

C:\Users\T.Santos18\Documents\workspace\textparsing\html\10335.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10336.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10337.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10338.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10339.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\1034.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10340.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10341.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10342.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10343.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10344.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10345.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10346.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10347.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\10448.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10449.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\1045.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10450.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10451.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10452.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10453.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10454.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10455.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10456.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10457.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10458.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\10459.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\1046.html
C:\Users\T.Santos18\Documents\workspace\textparsin

...

C:\Users\T.Santos18\Documents\workspace\textparsing\html\27627.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27628.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27629.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2763.html
exception:  19464  ###
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27631.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27632.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27633.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27634.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27635.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27636.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27637.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27638.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27639.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2764.html
C:\Users\T.Santos18\Documen

C:\Users\T.Santos18\Documents\workspace\textparsing\html\27738.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27739.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2774.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27740.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27741.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27742.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27743.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27744.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27745.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27746.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27747.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27748.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27749.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2775.html
C:\Users\T.Santos18\Documents\workspace\textparsin

C:\Users\T.Santos18\Documents\workspace\textparsing\html\27849.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2785.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27850.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27851.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27852.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27853.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27854.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27855.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27856.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27857.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27858.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27859.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2786.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27860.html
C:\Users\T.Santos18\Documents\workspace\textparsin

C:\Users\T.Santos18\Documents\workspace\textparsing\html\2796.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27960.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27961.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27962.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27963.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27964.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27965.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27966.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27967.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27968.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27969.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2797.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27970.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\27971.html
C:\Users\T.Santos18\Documents\workspace\textparsin

C:\Users\T.Santos18\Documents\workspace\textparsing\html\28072.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28073.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28074.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28075.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28076.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28077.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28078.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28079.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\2808.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28080.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28081.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28082.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28083.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\28084.html
C:\Users\T.Santos18\Documents\workspace\textparsi

In [58]:
from dateutil.parser import parse
dateformat = '%B %d, %Y %I:%M %p %Z'
temp_date = 'Sage Group plc ( OTC:SGGEF ) Q4 2014 Earnings Conference Call December 3, 2014 3:45 AM ET'

date_out = dateparser.parse(date_string=temp_date,settings={'DATE_ORDER': 'MDY', 'PREFER_LANGUAGE_DATE_ORDER': False})


In [231]:
date_out = arrow.get(temp_date,'MMMM D, YYYY H:mm A')

In [239]:
date_out.isoformat()

'2014-12-03T03:45:00+00:00'